In [1]:
import talib
import MetaTrader5 as mt
import pandas as pd
import plotly.express as px
import matplotlib.pylab as plt
import numpy as np
from datetime import datetime
from sklearn.model_selection import GridSearchCV
import talib
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, make_scorer
from sklearn.ensemble import RandomForestClassifier

ModuleNotFoundError: No module named 'MetaTrader5'

In [2]:
from talipp.indicators import EMA, SMA, Stoch, DPO
from talipp.ohlcv import OHLCVFactory

In [3]:
mt.initialize()
login= 51505377
password="7JhZp8Vj"
server="ICMarketsSC-Demo"
mt.login(login,password,server)

True

In [4]:
symbol="GBPUSD"
timeframe=mt.TIMEFRAME_D1

#ohcl data
ohlc_data=pd.DataFrame(mt.copy_rates_range(symbol,timeframe,datetime(2021,1,1),datetime.now()))
df=pd.DataFrame(ohlc_data)[['time','open','high','low','close']]

#ohlc_data=pd.DataFrame(mt.copy_rates_from_pos(symbol, timeframe, 1, 99100))
df_temp=pd.DataFrame(ohlc_data)[['time','open','high','low','close']]
#df['time']=pd.to_datetime(df['time'],unit='s')
#print(df['time'])

#SELL
# df['sma_5'+symbol]=df_temp['close'].rolling(5).mean()
# df['sma_5'+symbol]=df['sma_5'+symbol].shift(periods=1)
# df['sma_957'+symbol]=df_temp['close'].rolling(957).mean()
# df['sma_957'+symbol]=df['sma_957'+symbol].shift(periods=1)
# df['WILLR_45']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=45) #Williams' %R
# df['WILLR_45']=df['WILLR_45'].shift(periods=1)
# df['WILLR_82']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=82) #Williams' %R
# df['WILLR_82']=df['WILLR_82'].shift(periods=1)
# df['WILLR_1300']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=1300) #Williams' %R
# df['WILLR_1300']=df['WILLR_1300'].shift(periods=1)
# df['WILLR_1340']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=1340) #Williams' %R
# df['WILLR_1340']=df['WILLR_1340'].shift(periods=1)

#BUY
# df['sma_36'+symbol]=df_temp['close'].rolling(36).mean()
# df['sma_36'+symbol]=df['sma_36'+symbol].shift(periods=1)
# df['sma_890'+symbol]=df_temp['close'].rolling(890).mean()
# df['sma_890'+symbol]=df['sma_890'+symbol].shift(periods=1)

df['WILLR_15']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=15) #Williams' %R
df['WILLR_15']=df['WILLR_15'].shift(periods=1)
df['WILLR_23']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=23) #Williams' %R
df['WILLR_23']=df['WILLR_23'].shift(periods=1)
df['WILLR_42']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=42) #Williams' %R
df['WILLR_42']=df['WILLR_42'].shift(periods=1)
df['WILLR_145']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=145) #Williams' %R
df['WILLR_145']=df['WILLR_145'].shift(periods=1)


# df['WILLR_2']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=2) #Williams' %R
# df['WILLR_2']=df['WILLR_2'].shift(periods=1)
# df['WILLR_5']=talib.WILLR(df['high'],df['low'],df['close'],timeperiod=5) #Williams' %R
# df['WILLR_5']=df['WILLR_5'].shift(periods=1)

#df['mean_candle'] = 0
df['b_flag'] = 0
df['s_flag'] = 0
#df['mult1']=df['sma_20']*df['sma_30']
#df['mult2']=df['sma_20']*df['sma_90']
df=df.dropna()
df=df.reset_index(drop=True)

In [5]:
mean_candle=0
for i in range(len(df)):
    mean_candle= mean_candle + (df['high'][i]-df['low'][i])
mean_candle=mean_candle/len(df)
print(mean_candle)

0.013224896456511714


In [6]:
import warnings
warnings.filterwarnings('ignore')

mult1=[1]
mult2=[1.25]

loops=80
b_sum=0
s_sum=0
b_cols = ['b_sum','mean_candle','b_sl', 'b_tp','b_lost','b_win','b_nothing','loops']
b_data = pd.DataFrame(columns=b_cols)
s_cols = ['s_sum','mean_candle','s_sl', 's_tp','s_lost','s_win','s_nothing','loops']
s_data = pd.DataFrame(columns=s_cols)

for m1 in range(len(mult1)):
    b_sl = mean_candle*mult1[m1]
    s_sl = mean_candle*mult1[m1]
    for m2 in range(len(mult2)):
        b_tp = mean_candle*mult2[m2]
        s_tp = mean_candle*mult2[m2]
        for i in range(len(df)-loops):

            price = df['open'][i]
            price_b_sl = df['open'][i]-b_sl
            price_b_tp = df['open'][i]+b_tp
            price_s_sl = df['open'][i]+s_sl
            price_s_tp = df['open'][i]-s_tp

            for j in range(loops):

                if df['low'][i+j] < price_b_sl: 
                    df['b_flag'][i]=0
                    b_sum= b_sum - b_sl
                    break
                elif df['high'][i+j] > price_b_tp: 
                    df['b_flag'][i]=1
                    b_sum= b_sum + b_tp
                    break
                else: df['b_flag'][i]=0
            
            for j in range(loops):
                    
                if df['high'][i+j] > price_s_sl: 
                    df['s_flag'][i]=0
                    s_sum= s_sum - s_sl
                    break
                elif df['low'][i+j] < price_s_tp:
                    df['s_flag'][i]=1
                    s_sum= s_sum + s_tp
                    break
                else: 
                    df['s_flag'][i]=0

        b_raw_data = {'b_sum': [b_sum],'b_sl': [b_sl],'b_tp': [b_tp],'b_lost':[df['b_flag'].value_counts()[0]],
                      'b_win':[df['b_flag'].value_counts()[1]],'b_nothing':[df['b_flag'].value_counts()[0]],
                      'mean_candle':[mean_candle],'loops':[loops]}
        b_temp = pd.DataFrame(b_raw_data, columns=b_cols)
        b_data=b_data.append(b_temp) 
        b_sum=0
        
        s_raw_data = {'s_sum': [s_sum],'s_sl': [s_sl],'s_tp': [s_tp],'s_lost':[df['s_flag'].value_counts()[0]],
                      's_win':[df['s_flag'].value_counts()[1]],'s_nothing':[df['s_flag'].value_counts()[0]],
                      'mean_candle':[mean_candle],'loops':[loops]}
        s_temp = pd.DataFrame(s_raw_data, columns=s_cols)
        s_data=s_data.append(s_temp) 
        s_sum=0
        
print(b_data)
print(s_data)

          b_sum  mean_candle      b_sl      b_tp b_lost b_win b_nothing loops
0  1.371125e-14     0.013225  0.013225  0.016531   2450  1896      2450    80
      s_sum  mean_candle      s_sl      s_tp s_lost s_win s_nothing loops
0 -0.952193     0.013225  0.013225  0.016531   2482  1864      2482    80


In [7]:
print(df['b_flag'].value_counts())
print(df['s_flag'].value_counts())

0    2450
1    1896
Name: b_flag, dtype: int64
0    2482
1    1864
Name: s_flag, dtype: int64


In [8]:
df=df.drop(columns=['time','high','low','close','open'])
#df['open']=df['open'].shift(periods=1)
#df['high']=df['high'].shift(periods=1)
#df['low']=df['low'].shift(periods=1)
#df['close']=df['close'].shift(periods=1)
df=df.dropna()
df=df.reset_index(drop=True)

X=df.iloc[:,0:len(df.columns)-2].values
y=df.iloc[:,len(df.columns)-2].values        #Buy
#y=df.iloc[:,len(df.columns)-1].values       #Sell

X2=df.iloc[:,0:len(df.columns)-2]

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=0, shuffle=False)

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
#fig1=px.line(x_train,x=0,y=[1])
#fig1.show()
x_train=sc.fit_transform(x_train)
#fig2=px.line(x_train,x=0,y=[1])
#fig2.show()
x_test=sc.fit_transform(x_test)
#x_train
X=sc.fit_transform(X)

mF=4

In [ ]:
from sklearn.feature_selection import RFECV
rfecv = RFECV(estimator=RandomForestClassifier(max_features='auto',random_state=5, n_estimators= 80, class_weight= {0:5,1:6}),min_features_to_select=mF , step=1, cv=5, scoring='precision',n_jobs=-1,verbose=0)
rfecv=rfecv.fit(x_train,y_train)

print('Optimal number of features:',rfecv.n_features_)
print('Best features:', X2.columns[rfecv.support_])
X2.drop(X2.columns[~rfecv.support_], axis=1, inplace=True)

rfecv.grid_scores_

plt.figure()
plt.xlabel("Number of Features")
plt.ylabel("CV Score")
plt.plot(range(1,len(rfecv.grid_scores_)+1),rfecv.grid_scores_)

x_train=rfecv.transform(x_train)
x_test=rfecv.transform(x_test)


RF_model=RandomForestClassifier(max_features='auto',random_state=5, n_estimators= 80, class_weight= {0:5,1:6})
RF_rfecv_model=RF_model.fit(x_train,y_train)

y_pred=RF_rfecv_model.predict(x_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print(precision_score(y_test,y_pred))

In [10]:
# import joblib
# joblib.dump(RF_rfecv_model, 'model_EURUSD_D1_buy.joblib')

In [11]:
grid=GridSearchCV(
    estimator=RandomForestClassifier(random_state=5, n_estimators= 80,class_weight= {0:5,1:6}, max_features=None),
    #param_grid={'class_weight': [{0:5,1:v} for v in range(5,20,1)]},
    #param_grid={'class_weight': [{0:5,1:6}]},
    #param_grid={'n_estimators': [50,60,70,80,90,100,110,120,130,140,150]},
    param_grid={'n_estimators': [80]},
    #param_grid={'random_state': [1,2,3,4,5,6,7,8,9,10]},
    #param_grid={'max_features': ['auto']},
    scoring={'precision':make_scorer(precision_score)},
    #scoring={'balanced_accuracy_score':make_scorer(balanced_accuracy_score)},
    refit='precision',
    #return_train_score=True,
    cv=22,
    n_jobs=-1
)
grid.fit(x_train,y_train)
grid.n_features_in_

4

In [12]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(pd.DataFrame(grid.cv_results_))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_precision,split1_test_precision,split2_test_precision,split3_test_precision,split4_test_precision,split5_test_precision,split6_test_precision,split7_test_precision,split8_test_precision,split9_test_precision,split10_test_precision,split11_test_precision,split12_test_precision,split13_test_precision,split14_test_precision,split15_test_precision,split16_test_precision,split17_test_precision,split18_test_precision,split19_test_precision,split20_test_precision,split21_test_precision,mean_test_precision,std_test_precision,rank_test_precision
0,1.545332,0.20198,0.019129,0.013702,80,{'n_estimators': 80},0.538462,0.583333,0.479452,0.402778,0.444444,0.521739,0.575,0.517241,0.457627,0.569444,0.42029,0.6,0.458333,0.4375,0.465116,0.416667,0.42623,0.481481,0.292308,0.36,0.391892,0.507692,0.47032,0.075511,1


In [13]:
y_pred=grid.predict(x_test)
print(len(x_test))

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print(precision_score(y_test,y_pred))

1087
[[406 233]
 [265 183]]
              precision    recall  f1-score   support

           0       0.61      0.64      0.62       639
           1       0.44      0.41      0.42       448

    accuracy                           0.54      1087
   macro avg       0.52      0.52      0.52      1087
weighted avg       0.54      0.54      0.54      1087

0.43990384615384615


In [14]:
import joblib
joblib.dump(grid, r'C:\Users\Koch\Desktop\model_GBPUSD_D1_buy.joblib')

['C:\\Users\\Koch\\Desktop\\model_GBPUSD_D1_buy.joblib']